In [1]:
from transformers import pipeline
import spacy
import en_core_web_sm
import nltk
from goose3 import Goose
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge import Rouge

In [2]:
summarizor = pipeline('summarization', model = 'google/pegasus-cnn_dailymail')

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")

model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail")

In [4]:
tokenizer.model_max_length

1024

In [5]:
file = "C:\\Lachin\\NLP Project\\Full podcast.txt"
with open(file,'r') as f:
    original_text = f.readlines()
original_text = ''.join(original_text)

In [6]:
file1 = "C:\\Lachin\\NLP Project\\reference_summary.txt"
with open(file1,'r') as f1:
    reference_summary = f1.readlines()
reference_summary = ''.join(reference_summary)

In [7]:
def summarizing_text(text, number_of_tokens = tokenizer.model_max_length, max_len=0, min_len=0, human_summary=reference_summary):
    sentences = []
    total_sentences = []
    length = 0
    original_sentences = nltk.tokenize.sent_tokenize(text)
    for sent in original_sentences:
        word_tokens = tokenizer(sent)
        length += len(word_tokens['input_ids'])
        if (length < number_of_tokens):
            sentences.append(sent)
        else:
            total_sentences.append(sentences)
            length = len(word_tokens['input_ids'])
            sentences = []
            sentences.append(sent)
    
    
    #Summarizing the text now
    system_summary = []
    for i in range(len(total_sentences)):
        if (i==0):
            print(len(total_sentences))
        print(i)
        text1 = ' '.join(total_sentences[i])
        if ((max_len==0) and (min_len==0)):
            z = summarizor(text1, do_sample=False)
            system_summary.append(z[0]['summary_text'])
        elif((max_len >0) and (min_len > 0)):
            z = summarizor(text1, max_length=max_len, min_length =min_len, do_sample=False)
            system_summary.append(z[0]['summary_text'])
        else:
            print('Please check "max_len" and "min_len" parameters')
    final_system_summary = " ".join(system_summary)
    final_summary_tokens = tokenizer(final_system_summary)
    print('Number of tokens in system summary {}'.format(len(final_summary_tokens['input_ids'])))
    human_summary_tokens = tokenizer(human_summary)
    print('Number of tokens in human summary {}'.format(len(human_summary_tokens['input_ids'])))
    
    #Accuracy checking
    r = Rouge()
    results = r.get_scores(final_system_summary,human_summary)
    print(results)
    
    
    print(final_system_summary)

In [9]:
%%time
summarizing_text(original_text)

6
0
1
2
3
4
5
Number of tokens in system summary 458
Number of tokens in human summary 502
[{'rouge-1': {'r': 0.43891402714932126, 'p': 0.5359116022099447, 'f': 0.48258705972612065}, 'rouge-2': {'r': 0.2623376623376623, 'p': 0.3258064516129032, 'f': 0.29064747707261535}, 'rouge-l': {'r': 0.4253393665158371, 'p': 0.5193370165745856, 'f': 0.4676616865917923}}]
The most important part of starting a company is finding the right co-founder .<n>There are three main reasons you should have a co-founder .<n>Startups are an incredibly intense and taxing journey .<n>Apple, Facebook, Google, Microsoft, they all had co-founders when they started . The best single founders are also able to run a great background search for co-founders .<n>The single most important thing to know about someone before you start a company is how do they handle stress .<n>A great co-founder should be able to support you through those and help you through those . It's really important to talk with someone, even if you kn

In [10]:
%%time
summarizing_text(original_text, number_of_tokens=512)

14
0
1
2
3
4
5
6
7
8
9
10
11
12
13
Number of tokens in system summary 1011
Number of tokens in human summary 502
[{'rouge-1': {'r': 0.5610859728506787, 'p': 0.38509316770186336, 'f': 0.456721910458438}, 'rouge-2': {'r': 0.4051948051948052, 'p': 0.23853211009174313, 'f': 0.3002887345074344}, 'rouge-l': {'r': 0.5520361990950227, 'p': 0.37888198757763975, 'f': 0.44935542795383404}}]
Find the right co-founder is arguably the most important part of starting a company .<n> productivity, you can get a lot more work done if you have someone to divide the work up with .<n>The second reason is moral support. Startups are an incredibly intense and taxing journey . You can't be as open or honest with them about the company as you can be with a peer .<n>Apple, Facebook, Google, Microsoft, they all had co-founders when they started .<n>So it's always important to remember that when you're starting a company, and wondering whether you should have a co-founder or not . The best single founders are als

In [11]:
%%time
summarizing_text(original_text, number_of_tokens=512, max_len=50, min_len=25)

14
0
1
2
3
4
5
6
7
8
9
10
11
12
13
Number of tokens in system summary 672
Number of tokens in human summary 502
[{'rouge-1': {'r': 0.48868778280542985, 'p': 0.4444444444444444, 'f': 0.46551723639055065}, 'rouge-2': {'r': 0.33766233766233766, 'p': 0.28322440087145967, 'f': 0.30805686707635166}, 'rouge-l': {'r': 0.48868778280542985, 'p': 0.4444444444444444, 'f': 0.46551723639055065}}]
Find the right co-founder is arguably the most important part of starting a company .<n> productivity, you can get a lot more work done if you have someone to divide the work up with .<n>The second reason is moral support You can't be as open or honest with them about the company as you can be with a peer .<n>Apple, Facebook, Google, Microsoft, they all had co-founders when they started .<n>In those days, Y Combinator has funded single founders, and we have done so successfully .<n>The best single founders are also able to run a great background search for co-founders .<n>If the opportunity arises, bring on